
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement et l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, fourni par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible.![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche du changement climatique à Lyon : Création d'un jeu de données

Ce cahier d'exercices propose plusieurs voies d'exploration d'un jeu de données issues des simulations climatiques de Météofrance. Le lot utilisé  est issu d'une série de modélisations  RCP 2.6 - 4.5 - 8.5. Le travail a été effectué avec la température maximale mais il est directement transposable à toutes les variables du modèle (température min, humidité. Cette première partie traite de la méhode pour qu'un enseignant puisse extraire les données dont il a besoin d'un fichier commandé sur le site [DRIAS](http://www.drias-climat.fr/). Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Ce cahier utilise les données d'un carré 10 x noeuds de la grille ALADIN centré sur la ville de Lyon. 

## Préparation de l'environnement et ouverture du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [5]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, datetime, os

Importation des données de températures maximales depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise et intégration dans un fichier pour le traitement, puis affichage de la description du contenu, de la liste des variables.
L'exemple choisi ici a été réalisé avec une grille de 10 x 10 noeuds centrés sur la ville de Lyon, pour obtenir un jeu de données se reporter au manuel numérique réalisé par E. Le Jan et CArole Larose dans le cadre du projet "Climat et Météo Tremplin pour l'enseignement des sciences". Les deux affichages proposés permettent de vérifier les propriétés  du fichier obtenu ainsi que les variables qui pourront être utilisées. Ces affichages sont facultatifs et peuvent être commentés sans conséquence pour la suite.

In [20]:
# Attention Avant de relire les sources en ligne, si des fichiers locaux sont OUVERTS
# il est nécessaire de les  fermer pour éviter des erreurs de données
# Close Datasets if necessary
#try: os.remove('tMax_00_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
#except OSError : pass
try: os.remove('tMax_26_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_45_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_85_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass


In [21]:
# Lecture des fichiers distants
# Attention Avant de relire/recharger les sources en ligne il est IMPÉRATIF de fermer les fichiers locaux
#
#tMax_00_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc')
tMax_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc')
tMax_45_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc')
tMax_85_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc')
#print('Description des données issues du modèle : \n',tMax_26_Lyon,'\n') 
print('Variables disponibles :',tMax_26_Lyon.variables.keys()) # get all variable names
print(tMax_26_Lyon.variables['tasmax'].units)
print('Taille du tableau tasmax :',tMax_26_Lyon.variables['tasmax'].shape ,'\n')


Variables disponibles : odict_keys(['i', 'j', 'time', 'lat', 'lon', 'tasmax', 'x', 'y'])
K
Taille du tableau tasmax : (34698, 10, 10) 



## Liste des dimensions et des variables du système de données

À partir de la liste des variables obtenue ci-dessus on renomme les jeux de données de chacune des variables qui seront exploitées apour effectuer les calculs et contrôle de la taille des échantillons. Les affichages proposés permettent de contrôler que les paramètres présents sont effectivement ceux qui sont attendus.

In [23]:
#lyon_temp_00 = np.zeros_like(tMax_00_Lyon.variables['tasmax'])
lyon_temp_26 = np.empty_like(tMax_26_Lyon.variables['tasmax'])
lyon_temp_45 = np.empty_like(tMax_45_Lyon.variables['tasmax'])
lyon_temp_85 = np.empty_like(tMax_85_Lyon.variables['tasmax'])
lyon_date = np.empty_like(tMax_26_Lyon.variables['time'])
lyon_lat,lyon_lon = np.zeros_like(tMax_26_Lyon.variables['lat']), np.zeros_like(tMax_26_Lyon.variables['lon'])  # latitude longitude
lyon_x,lyon_y = np.zeros_like(tMax_26_Lyon.variables['x']), np.zeros_like(tMax_26_Lyon.variables['y'])  # coordonnées métriques
lyon_gridi,lyon_gridj = np.zeros_like(tMax_26_Lyon.variables['i']), np.zeros_like(tMax_26_Lyon.variables['j']) # coordonnées grille Aladin
print(lyon_temp_26.shape)
for dim in tMax_26_Lyon.dimensions.items():
    print(dim[1])
#lyon_temp_00 = tMax_00_Lyon.variables['tasmax']  # variable temperature 
np.copyto(lyon_temp_26,tMax_26_Lyon.variables['tasmax'][:]) # variable temperature 
lyon_temp_26.units = tMax_26_Lyon.variables['tasmax'].units
print('\n',tMax_26_Lyon.variables['tasmax'][11113,:,:])
print('\n',lyon_temp_26[11113,:,:])
lyon_temp_45 = tMax_45_Lyon.variables['tasmax']  # variable temperature 
lyon_temp_85 = tMax_85_Lyon.variables['tasmax'] # variable temperature 
lyon_date = tMax_26_Lyon.variables['time']  # variable temps
#Sans ce test le peut programme peut engendrer des erreurs silencieuses
print(np.any(np.any(np.isnan(lyon_temp_26), axis=0)))
if (np.any(np.any(np.isnan(lyon_temp_26), axis=0))) :
    print ('Problem loading file 26 from server')
if (np.any(np.any(np.isnan(lyon_temp_26), axis=0))) :
    print ('Problem loading file 45 from server')
if (np.any(np.any(np.isnan(lyon_temp_26), axis=0))) :
    print ('Problem loading file 85 from server')
lyon_lat,lyon_lon = tMax_26_Lyon.variables['lat'], tMax_26_Lyon.variables['lon']  # latitude longitude
lyon_x,lyon_y = tMax_26_Lyon.variables['x'], tMax_26_Lyon.variables['y']  # coordonnées métriques
lyon_gridi,lyon_gridj = tMax_26_Lyon.variables['i'], tMax_26_Lyon.variables['j'] # coordonnées grille Aladin
#
print ('Variables \t  Forme \t\t Taille \t type :  \n')
for var in tMax_26_Lyon.variables.keys() :
    print (var, '\t\t', tMax_26_Lyon.variables[var].dimensions, '\t\t', tMax_26_Lyon.variables[var].shape, '\t', tMax_26_Lyon.variables[var].dtype)
print ('\n Unités : \n', lyon_temp_26.units, '\t', 
      lyon_date.units, '\t',
      lyon_lat.units, '\t', 
      lyon_lon.units, '\t',
      lyon_x.units, '\t', 
      lyon_y.units, '\n')
#print(lyon_gridi[:])
#print(lyon_gridj[:])
#print(lyon_temp_26[0,:,:])
print(lyon_temp_26.shape)
print(lyon_temp_26[0,0,0])
print(lyon_date[1])
print(lyon_temp_26[2000,:,:])
print(lyon_temp_45[2400,:,:])
print(tMax_26_Lyon.variables['tasmax'][2000,5,8])
print(tMax_26_Lyon.variables['time'].shape)
#print(tMax_26_Lyon.variables['time'] [-1])

(34698, 10, 10)
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 34698

<class 'netCDF4._netCDF4.Dimension'>: name = 'i', size = 10

<class 'netCDF4._netCDF4.Dimension'>: name = 'j', size = 10


 [[298.14014 297.37186 297.2022  297.20062 298.88126 298.34256 297.64523
  297.94318 298.15988 298.03247]
 [297.44937 296.59683 298.50327 297.63004 298.01355 297.50244 297.5481
  298.82648 297.9359  297.43518]
 [296.38547 295.6447  296.55838 298.8243  298.74173 297.75742 297.7909
  298.90253 298.06815 298.37665]
 [297.4315  297.14133 296.89603 298.70303 298.19122 297.71567 297.15833
  298.0509  299.26358 299.2017 ]
 [297.19424 297.39465 296.7592  298.20877 298.6805  297.13968 296.8198
  297.71646 298.62256 298.11316]
 [296.5952  296.60577 298.20877 298.1895  298.19138 297.04135 297.08533
  298.31256 298.83102 298.25153]
 [294.4456  296.24625 297.0201  297.6787  297.43372 297.52066 297.4086
  298.3204  298.0136  297.97772]
 [295.05902 296.2863  296.72903 297.5119  297.9484

In [9]:
print(tMax_26_Lyon.variables['time'] [-1])
print(lyon_date[0],lyon_date[-1],lyon_date[34697])

1324368.0
491640.0 1324368.0 1324368.0


## Construction du jeu de données de sortie des moyennes en fonction de : année mois latitude longitude

Création du fichier de sortie au format netCDF où seront stockées les valeurs moyennes calculées pour chaque noeud

In [10]:
try: os.remove('tsmaxLyon-1.nc')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
extractLyonTempYearMonth = nc.Dataset('tsmaxLyon-1.nc',mode='w',format='NETCDF4') 

Définition et affectation des variables où sont copiées les données conservées et où seront stockés les résultats des calculs. (La syntaxe du fichier netCDF reste à  vérifier). Les années seront calculées pendant le calcul principal, les affichages permettent de vérifier la validité des données utilisées.

In [11]:
listmonth = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'total']
lenMonthA =[31,28,31,30,31,30,31,31,30,31,30,31]
lenMonthB =[31,29,31,30,31,30,31,31,30,31,30,31]
firstyear = int(nc.num2date(lyon_date[0],lyon_date.units).strftime("%Y"))
lastyear = int(nc.num2date(lyon_date[-1],lyon_date.units).strftime("%Y"))
sizemonths = len(listmonth)
sizeyears = lastyear - firstyear + 1
sizegridi = lyon_gridi.shape[0]
sizegridj = lyon_gridj.shape[0]
print ("DateTime de départ  de l'étude : ", firstyear,
      "\nDateTime de fin  de l'étude : ", lastyear,
      "\nDurée de l'étude : ", sizeyears, 'ans')
extractLyonTempYearMonth.createDimension('i', sizegridi)     # latitude axis
extractLyonTempYearMonth.createDimension('j', sizegridj)    # longitude axis
extractLyonTempYearMonth.createDimension('month', sizemonths)    # month axis
extractLyonTempYearMonth.createDimension('year', sizeyears) # year axis
# Define two variables with the same names as dimensions,
# a conventional way to define "coordinate variables".
i = extractLyonTempYearMonth.createVariable('i', 'i4', ('i',))
i.long_name = 'cell index along first dimension'
i[:] = lyon_gridi[:]
print(i)
j = extractLyonTempYearMonth.createVariable('j', 'i4', ('j',))
j.long_name = 'cell index along second dimension'
j[:] = lyon_gridj[:]
lat = extractLyonTempYearMonth.createVariable('lat', np.float32, ('j','i'))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lat.standard_name = 'latitude'
lat._CoordinateAxisType = 'Lat'
lat[:,:] = lyon_lat[:,:]
lon = extractLyonTempYearMonth.createVariable('lon', np.float32, ('j','i',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
lon.standard_name = 'longitude'
lon._CoordinateAxisType = 'Lon'
lon[:,:] = lyon_lon[:,:]
month = extractLyonTempYearMonth.createVariable('month','c', ('month',))
month.units = 'month'
month.long_name = 'Name_of_the_month'
month.standard_name = 'Name_of_the_month'
month[:] = listmonth[:]
year = extractLyonTempYearMonth.createVariable('year', 'u4', ('year',))
year.units = 'date'
year.long_name = 'year'
year.standard_name = 'year'
year = np.arange(firstyear,lastyear+1)
print(year)
# Define a 3D variable to hold the data
#tempMY = extractLyonTempYearMonth.createVariable('tempMY',np.float64,('year','month','j','i')) # note: unlimited dimension is leftmost
#temp_00 = extractLyonTempYearMonth.createVariable('temp_00',np.float64,('year','month','j','i'))
#temp_00.units = '°C' # degrees Kelvin
#temp_00.standard_name = 'air_temperature' # this is a CF standard name
temp_26 = extractLyonTempYearMonth.createVariable('temp_26',np.float32,('year','month','j','i'))
temp_26.units = 'degree C' # degrees Kelvin
temp_26.standard_name = 'air_temperature_26' # this is a CF standard name
temp_45 = extractLyonTempYearMonth.createVariable('temp_45',np.float32,('year','month','j','i'))
temp_45.units = 'degree C' # degrees Kelvin
temp_45.standard_name = 'air_temperature_45' # this is a CF standard name
temp_85 = extractLyonTempYearMonth.createVariable('temp_85',np.float32,('year','month','j','i'))
temp_85.units = 'degree C' # degrees Kelvin
temp_85.standard_name = 'air_temperature-85' # this is a CF standard name
extractLyonTempYearMonth.title = 'Extrait TSMax par moyenne mensuelle de 2006 a 2100 Lyon et sa region'
extractLyonTempYearMonth.institution = 'ENS de Lyon'
extractLyonTempYearMonth.institute_id = 'IFE Institut Francais de l Education'
extractLyonTempYearMonth.project_id = 'Climat et meteo tremplin pour l enseignement des sciences'
extractLyonTempYearMonth.model_id = 'CNRM-ALADIN52'
extractLyonTempYearMonth.product = 'output derived from Meteofrance DRIAS data'
extractLyonTempYearMonth.contact = 'gerard.vidal@ens-lyon.fr'
extractLyonTempYearMonth.creation_date = str(datetime.datetime.now())
extractLyonTempYearMonth.driving_experiment_name = 'DRIAS2014'
extractLyonTempYearMonth.experiment = 'RCP2.6 RCP4.5 RCP8.5 '
extractLyonTempYearMonth.model = 'ALADIN-Climat'
extractLyonTempYearMonth.author = 'Gerard Vidal'
extractLyonTempYearMonth.comment = "Extraction des moyennes de la region Lyonnaise de 2006 a 2100 \
                                    et changegement des variables"
#print(gridi[:],gridj[:])
#print(gridi.shape,gridj.shape)
#print(lat[2,:])

DateTime de départ  de l'étude :  2006 
DateTime de fin  de l'étude :  2100 
Durée de l'étude :  95 ans
<class 'netCDF4._netCDF4.Variable'>
int32 i(i)
    long_name: cell index along first dimension
unlimited dimensions: 
current shape = (10,)
filling on, default _FillValue of -2147483647 used

[2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033
 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047
 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061
 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075
 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087 2088 2089
 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100]


## Calcul principal des moyennes par mois pour chaque noeud et toutes les années


In [12]:
iteri = 0
iterj = 0
iteriFirst = iteri  
while iteri  < lyon_temp_26.shape[0] :      
    for iterj in  range(sizeyears) :
#        year[iterj] = (nc.num2date(lyon_date[iteri],lyon_date.units).strftime("%Y"))
#        print(nc.num2date(lyon_date[iteri],lyon_date.units), iteri, iterj)  # vérification du point de départ
        print(iterj,year[iterj] )
        if (iterj % 4 == 2) :
            for p in range(len(lenMonthB)) :
                iteriLast = iteri + lenMonthB[p]
                # moyenne du mois
#                temp_00[iterj,p,:,:] = np.mean(lyon_temp_00[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
        else :
            for p in range(len(lenMonthA)) :
                iteriLast = iteri + lenMonthA[p]
                # moyenne du mois
#                temp_00[iterj,p,:,:] = np.mean(lyon_temp_00[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
       # moyenne de l'année
#        temp_00[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_00[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_26[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_26[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_45[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_45[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_85[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_85[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        iteriFirst = iteriLast+1
#print(iteri)
#print('fin du traitement : ', nc.num2date(lyon_date[iteri-2],lyon_date.units)) 
# vérification de la fin d'année   
#years = year[:].tolist()
#print ('years : \n', years)
#extractLyonTempYearMonth.close() # Ne pas oublier de fermer le fichier netCDF
print (temp_26.dtype, temp_45.dtype, temp_85.dtype)

0 2006
1 2007
2 2008
3 2009
4 2010
5 2011
6 2012
7 2013
8 2014
9 2015
10 2016
11 2017
12 2018
13 2019
14 2020
15 2021
16 2022
17 2023
18 2024
19 2025
20 2026
21 2027
22 2028
23 2029
24 2030
25 2031
26 2032
27 2033
28 2034
29 2035
30 2036
31 2037
32 2038
33 2039
34 2040
35 2041
36 2042
37 2043
38 2044
39 2045
40 2046
41 2047
42 2048
43 2049
44 2050
45 2051
46 2052
47 2053
48 2054
49 2055
50 2056
51 2057
52 2058
53 2059
54 2060
55 2061
56 2062
57 2063
58 2064
59 2065
60 2066
61 2067
62 2068
63 2069
64 2070
65 2071
66 2072
67 2073
68 2074
69 2075
70 2076
71 2077
72 2078
73 2079
74 2080
75 2081
76 2082
77 2083
78 2084
79 2085
80 2086
81 2087
82 2088
83 2089
84 2090
85 2091
86 2092
87 2093
88 2094
89 2095
90 2096
91 2097
92 2098
93 2099
94 2100
float32 float32 float32


Les affichages suivants permettent de vérifier  que les données obtenues correspondent au format attendu

In [23]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])

In [24]:
#for dim in extractLyonTempYearMonth.dimensions.items():
#    print(dim[1])
#for var in extractLyonTempYearMonth.variables.keys() :
#    print (var, '\t\t', extractLyonTempYearMonth.variables[var].dimensions, '\t\t', 
#           extractLyonTempYearMonth.variables[var].shape, '\t', extractLyonTempYearMonth.variables[var].dtype)

In [13]:
extractLyonTempYearMonth.close()